In [43]:
import pandas as pd
import ast 
import numpy as np

In [44]:
data = pd.read_csv('data/extracted_data_final_0_1000.csv')
data.head()

,Doctor Name,Specialty,Map Location,Skills,Profile Cards,URL
0,Dr Marian AGACHI,Neurochirurgien,"{""lat"":48.8783278,""lng"":2.431033999999954,""ful...",Expertises et actes\nNeurochirurgie du rachis,{'card_0': 'Centre Floréal\nLieu 2\nLieu 3\nCe...,https://www.doctolib.fr/neurochirurgien/paris/...
1,Dr Anne Vaillant Moga,Médecin généraliste,"{""lat"":48.8692182,""lng"":2.285468,""fullAddress""...",Expertises et actes\nAcné\nAllergie\nApnée du ...,"{'card_0': ""Lieu 1\nLieu 2\nDr Anne Moga (Pari...",https://www.doctolib.fr/medecin-generaliste/pa...
2,Dr Gabriel Ohana,Chirurgien-dentiste,"{""lat"":48.8941478,""lng"":2.2506978,""fullAddress...",Expertises et actes\nChirurgie buccale\nProthè...,{'card_0': 'Tarifs et remboursement\n\nCarte V...,https://www.doctolib.fr/dentiste/courbevoie/ga...
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"{""lat"":48.8154325,""lng"":2.3430171,""fullAddress...",Skills missing,{'card_0': 'Tarifs et remboursement\n\n\nConve...,https://www.doctolib.fr/dermatologue/gentilly/...
4,Dr Layla Chatila,Chirurgien-dentiste,"{""lat"":48.8588045,""lng"":2.274392,""fullAddress""...",Skills missing,{'card_0': 'Tarifs et remboursement\n\n\n\nCon...,https://www.doctolib.fr/dentiste/paris/layla-c...


In [45]:
# Fonction pour réorganiser les profils cards
def reorder_cards(profile_cards):
    try:
        cards_dict = ast.literal_eval(profile_cards)
        
        reordered_cards = {
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

        for key, value in cards_dict.items():
            if value.startswith("Carte et informations d'accès"):
                reordered_cards['card_1'] = value.split('\n') 
            elif value.startswith("Présentation"):
                reordered_cards['card_2'] = value.split('\n') 
            elif value.startswith("Horaires et coordonnées"):
                reordered_cards['card_3'] = value.split('\n') 
            elif value.startswith("Tarifs"):
                reordered_cards['card_4'] = value.split('\n') 
            elif value.startswith("Informations légales"):
                reordered_cards['card_5'] = value.split('\n') 
        
        return reordered_cards
    except:
        return {
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

reordered_cards = data['Profile Cards'].apply(reorder_cards)

reordered_cards_df = pd.json_normalize(reordered_cards)

data_combined = pd.concat([data, reordered_cards_df], axis=1)

data_combined.drop(columns = 'Profile Cards', inplace=True)

print(data_combined[['Doctor Name', 'card_1', 'card_2', 'card_3', 'card_4', 'card_5']].head())

             Doctor Name                                             card_1  \
0       Dr Marian AGACHI  [Carte et informations d'accès, Centre médico-...   
1  Dr Anne Vaillant Moga  [Carte et informations d'accès, Dr Anne Moga (...   
2       Dr Gabriel Ohana  [Carte et informations d'accès, Cabinet dentai...   
3    Dr Thomas BAMBERGER  [Carte et informations d'accès, Villa Amélie, ...   
4       Dr Layla Chatila  [Carte et informations d'accès, 5 rue François...   

                                              card_2  \
0  [Présentation, Le docteur Marian Agachi vous a...   
1  [Présentation, Située au cœur du  seizieme arr...   
2  [Présentation, Le docteur Gabriel Ohana vous a...   
3  [Présentation, Le docteur Thomas Bamberger vou...   
4  [Présentation, Le docteur Layla Chatila vous a...   

                                              card_3  \
0  [Horaires et coordonnées, Horaires d'ouverture...   
1  [Horaires et coordonnées, Horaires d'ouverture...   
2  [Horaires et coor

In [46]:
#Pour rendre la localisation plus claire
def extract_lat_lng(map_location):
    try:
        location_dict = ast.literal_eval(map_location)
        return (location_dict['lat'], location_dict['lng'])
    except:
        return (None, None)

data_combined['Latitude'], data_combined['Longitude'] = zip(*data_combined['Map Location'].apply(extract_lat_lng))

data_combined.drop(columns = 'Map Location', inplace=True)

print(data_combined.head())


             Doctor Name                    Specialty  \
0       Dr Marian AGACHI              Neurochirurgien   
1  Dr Anne Vaillant Moga          Médecin généraliste   
2       Dr Gabriel Ohana          Chirurgien-dentiste   
3    Dr Thomas BAMBERGER  Dermatologue et vénérologue   
4       Dr Layla Chatila          Chirurgien-dentiste   

                                              Skills  \
0      Expertises et actes\nNeurochirurgie du rachis   
1  Expertises et actes\nAcné\nAllergie\nApnée du ...   
2  Expertises et actes\nChirurgie buccale\nProthè...   
3                                     Skills missing   
4                                     Skills missing   

                                                 URL  \
0  https://www.doctolib.fr/neurochirurgien/paris/...   
1  https://www.doctolib.fr/medecin-generaliste/pa...   
2  https://www.doctolib.fr/dentiste/courbevoie/ga...   
3  https://www.doctolib.fr/dermatologue/gentilly/...   
4  https://www.doctolib.fr/dentiste/pari

In [47]:
# Fonction pour traiter la colonne 'Skills'
def process_skills(skills):
    if skills == 'Skills missing':
        return np.nan
    else:
        return skills.split('\n')

data_combined['Skills Processed'] = data_combined['Skills'].apply(process_skills)

data_combined.drop(columns = 'Skills', inplace=True)

print(data_combined.head())

             Doctor Name                    Specialty  \
0       Dr Marian AGACHI              Neurochirurgien   
1  Dr Anne Vaillant Moga          Médecin généraliste   
2       Dr Gabriel Ohana          Chirurgien-dentiste   
3    Dr Thomas BAMBERGER  Dermatologue et vénérologue   
4       Dr Layla Chatila          Chirurgien-dentiste   

                                                 URL  \
0  https://www.doctolib.fr/neurochirurgien/paris/...   
1  https://www.doctolib.fr/medecin-generaliste/pa...   
2  https://www.doctolib.fr/dentiste/courbevoie/ga...   
3  https://www.doctolib.fr/dermatologue/gentilly/...   
4  https://www.doctolib.fr/dentiste/paris/layla-c...   

                                              card_1  \
0  [Carte et informations d'accès, Centre médico-...   
1  [Carte et informations d'accès, Dr Anne Moga (...   
2  [Carte et informations d'accès, Cabinet dentai...   
3  [Carte et informations d'accès, Villa Amélie, ...   
4  [Carte et informations d'accès, 5 rue

In [48]:
data_combined['Specialty'].value_counts()

Specialty
Chirurgien-dentiste                                                 152
Médecin généraliste                                                 104
Ophtalmologue                                                        75
Cabinet médical                                                      56
Spécialiste en chirurgie plastique reconstructrice et esthétique     40
                                                                   ... 
Chirurgien oral                                                       1
Acupuncteur                                                           1
Addictologue                                                          1
Chirurgien cancérologue                                               1
Diététicien                                                           1
Name: count, Length: 78, dtype: int64

In [51]:
data_combined['RPPS'] = data_combined['card_5'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else np.nan)

data_combined.drop(columns = 'card_5', inplace = True)
print(data_combined.head())


             Doctor Name                    Specialty  \
0       Dr Marian AGACHI              Neurochirurgien   
1  Dr Anne Vaillant Moga          Médecin généraliste   
2       Dr Gabriel Ohana          Chirurgien-dentiste   
3    Dr Thomas BAMBERGER  Dermatologue et vénérologue   
4       Dr Layla Chatila          Chirurgien-dentiste   

                                                 URL  \
0  https://www.doctolib.fr/neurochirurgien/paris/...   
1  https://www.doctolib.fr/medecin-generaliste/pa...   
2  https://www.doctolib.fr/dentiste/courbevoie/ga...   
3  https://www.doctolib.fr/dermatologue/gentilly/...   
4  https://www.doctolib.fr/dentiste/paris/layla-c...   

                                              card_1  \
0  [Carte et informations d'accès, Centre médico-...   
1  [Carte et informations d'accès, Dr Anne Moga (...   
2  [Carte et informations d'accès, Cabinet dentai...   
3  [Carte et informations d'accès, Villa Amélie, ...   
4  [Carte et informations d'accès, 5 rue

In [55]:
data_combined['Nb Skills'] = data_combined['Skills Processed'].apply(lambda x: len(x)-1 if isinstance(x, list) else np.nan)

print(data_combined['card_3'].iloc[1])

['Horaires et coordonnées', "Horaires d'ouverture", "Aujourd'hui :", '10h30 - 19h00', "Contact d'urgence", "En cas d'urgence, contactez le 15 (Samu)"]


Ouverture annuaire santé

In [37]:
with open('data/PS_LibreAcces_Personne_activite.txt', 'r', encoding='utf-8') as file:
    contenu = file.read()

print(contenu)